In [ ]:
from collections import defaultdict
import gzip
import pickle
import codecs
import matplotlib.pyplot as plt
from pyodesys.symbolic import SymbolicSys, symmetricsys, get_logexp
from _chem_kinet import get_odesys
%matplotlib inline

In [ ]:
data = """
H4sIALeGpVgC/42YC1BUVRjH9wICvspHRlo+W+VDXTDQwNSFe4XO+mortTZ1pQUvKiFwAMUgFMjI
R6agkvIQTbOyNPKVmqSphNBML4acsUZn7DE+ZiRrJqeXdPfcC9zdvYfP2dmZe875nf/57nfP+f57
t9C/gJrA0cNkMi1NdyXnJGbJrmQqOKkfOKk/zDDN8FO+hh/ZSQOAN6gB3TAg0A0IXQBBmEIwBnTH
gB4Y0BPLQy9MoTcG3IcB92NAHwzoiwH9MKA/BjyAAQOwx/2gp4LgA4RgSzyEAQMxYBAGPIwBj2DA
YAwYggFDMWAYBgzHgBEY8CgGmDFgJAaMwoBQDAAMCMOA0RgwBgPGYoAFA8IxIAIDxmHAYxgQiQFR
GDAeAyZgwOMYEI0BMV0DsqOn4oVprhx5ZWK6a5mcTSc66RPAes3ywnxLAbhomJlOcgS7e2yJ+ZEF
djOdrDbtNjdhplMcQR2jZmp1+LOWmcY6AlTMTOMcvToFVE5UZ2ktSV2TtVTVqdqaGhCvKeiIBG3d
hfljlNaTmp46Rjr0ojTaJrNwmOVPc9LpiuXPAL7RsfTN7EyfnyEwS59fPwPgKego64Khgh10A4IB
8DToho2AZ8Cj29dFngWvTsELmA0+woIHMAcMohd0wFzw6PaN4Tnwmucdw/NgsLZ+CYcv4BnkC+AR
nG8M88Anek+F+WBwe/oYFhgdJz8d4ASDNOkVFmKWnwiGw0IH8CJfQWCAy/jMCx1AEnB3pAokYz9c
FgF/PgNk7DZTeEG2KywG3qPWgCXAj4ABS4EfAQNSecVR0PLwEvA2rKaQxk+UqrAMOE+iXSEdOMPt
ecgwSrVeIRP4wwygwM8CA7KAfw8MyAbeydaAHDCqTTpgOfDzyIAVwC8fDMgFboVSU72yK6sTFOBl
zCzzMCAfA17BgAIMWIUBqzGgELN8twtmZmUsokVOWqy44KuglTGBo7iGXy9U4DXstJcAz7804HXg
L8CAtVhJWoed1fXAP4gM2AAe5dz3Lt7gF1YV2Aj8isWAN7FysAn7PbEZi6HUKFH6JcqAZ7IasAV8
bcUD2Ao8g9OAbd612xso9/6PwRt4Cytq27GyuAOM7FEHVGBFrRIzuSpsT1ZjJrcTOzg1mMntwmxy
N2Zyb2M2uQczub3Y0XsHM7l92Ll4F9sP72H74X3M5PZjNvkBZnIfYjvqAGZyBzGb/AgzuVrMJj/m
7wd1iUN8BRU4jG37I1gmj2KGcgz4CzDgE+zgHO96V8uObooLqq+8J5z0JDgClbZscb/u0k/ZG6wt
0k5PsSu7zULrHH6sj37mENwX9DTrsNvoGWaoCh1JP1f7IulZdV6khZ5TtZS+8yrn7qxXxcbQL9QJ
FtqgToiy0AsyCyXTleValk0bnbQJiPhPzzUlm/KqifhH6KqWorImIt5u7T784JVNRBp4QjpGqvOI
NGxD/dKh5eVECkhccPZA1hQidV9QMND+s51IA+YKw78JbCNScLjLFLbH0nkhFi3++/uyqiNKz/yU
edHJ44jUbXzGivMjpc4LMf9i49WSPr0V5cyqjY3f7ieSKTfmqx1flxAxbHdr29YfK4m4ffDg3T9E
BRBrUerMC9evjCXirVlpKQ1Tr+ouGjJWHb597ToRdxbKv8YULifWv36ZUD7nUB2RYtsag5JqT5HY
4yNOrlsSr4QRmnTtz0lHbymBNay/s3/IcSJGpY88U1M37Z4YMXRranNpaIIBE3vup9TZvSwtJC5k
TvTEiH1HidS/uHJv/2hCxNyEO03ZV0tJXETh6r6TdwCJDfq3/pD/jRoiFl/ql5tScYeIN2/WbLE1
7SNxe2lLYXPrJWL9r3nz2km71pIpv9fduBsyYQ05HfhbraUixkSsl2LiB01PqCfW79oubF6Zk9oJ
Wy9fHuYcdXEbsd7NGl37ZXCQvDz8f4KclQygGAAA"""

In [ ]:
imported = pickle.loads(gzip.decompress(codecs.decode(data.encode('utf-8'), 'base64')))

In [ ]:
linsys = get_odesys(imported['reac'], imported['prod'], imported['names'],
                    SymbolicSys, latex_names=imported['latex_names'], dep_by_name=True)

In [ ]:
def integrate_and_plot(odesys):
    res = odesys.integrate(1e6, defaultdict(float, {'H2O': 55.4, 'H+': 1e-7, 'OH-': 1e-7, 'H2O2': [0, 1e-3, 1e-4]}),
                           imported['params'], integrator='cvode')
    fig, axes = plt.subplots(1, len(res), figsize=(16, 5))
    for r, ax in zip(res, axes):
        r.plot(xscale='log', yscale='log', ax=ax, legend=dict(prop={'size': 8}))

In [ ]:
integrate_and_plot(linsys)

In [ ]:
LogLogSys = symmetricsys(get_logexp(1, 1e-24), get_logexp(1, 1e-12))
tsys = LogLogSys.from_other(odesys)

In [ ]:
# would neeed to tile in pre_process:
#integrate_and_plot(tsys)